# Exercises: Chapter 2

# 0. Load and process data

In [1]:
import numpy as np
import pandas as pd

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.svm import SVR

from multiprocessing import cpu_count
from scipy.stats import expon, reciprocal

In [2]:
def load_housing_data():
    housing_path = 'datasets/housing/housing.csv'
    return pd.read_csv(housing_path)

In [3]:
housing = load_housing_data()
income_cat = pd.cut(housing["median_income"], bins=[0., 1.5, 3.0, 4.5, 6., np.inf], labels=[1, 2, 3, 4, 5])
y = housing["median_house_value"].values
X = housing.drop('median_house_value', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=income_cat)

In [4]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16512 entries, 17606 to 15775
Data columns (total 9 columns):
longitude             16512 non-null float64
latitude              16512 non-null float64
housing_median_age    16512 non-null float64
total_rooms           16512 non-null float64
total_bedrooms        16354 non-null float64
population            16512 non-null float64
households            16512 non-null float64
median_income         16512 non-null float64
ocean_proximity       16512 non-null object
dtypes: float64(8), object(1)
memory usage: 1.3+ MB


In [5]:
class TypeSelector(BaseEstimator, TransformerMixin):
    def __init__(self, dtype):
        self.dtype = dtype
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        assert isinstance(X, pd.DataFrame)
        return X.select_dtypes(include=self.dtype)

In [6]:
# column index
rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_rooms_per_household=True, add_population_per_household=True,
                 add_bedrooms_per_room=True):
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self  # nothing else to do
    def transform(self, X):
        if add_rooms_per_household:
            rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
            X = np.c_[X,rooms_per_household]
        if add_population_per_household:
            population_per_household = X[:, population_ix] / X[:, households_ix]
            X = np.c_[X,population_per_household]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            X = np.c_[X,bedrooms_per_room]
        return X

In [50]:
X = np.array([[1,2,3],[4,5,6]])

In [53]:
np.c_[X, []]

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 2 and the array at index 1 has size 0

In [7]:
pipeline_num = Pipeline([
    ('selector', TypeSelector('float')),
    ('imputer', SimpleImputer(strategy='median')),
    ('attr_adder', CombinedAttributesAdder()),
    ('scaler', RobustScaler())
])
pipeline_cat = Pipeline([
    ('selector', TypeSelector('object')),
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder())
])
pipeline_X = FeatureUnion([('numeric', pipeline_num), ('categorical', pipeline_cat)])

In [8]:
n_jobs = cpu_count() - 1
print(n_jobs)

11


## Exercise 1

Try a Support Vector Machine regressor ( sklearn.svm.SVR ) with various hyperparameters, such as kernel="linear" (with various values for the C hyperparameter) or kernel="rbf" (with various values for the C and gamma hyperparameters). Don’t worry about what these hyperparameters mean for now. How does the best SVR predictor perform?

### Wrong approach

Running the pipeline prior to cross-validation underestimates the generalization error and overestimates the generalization performance of a model, as statistics from the validation data leak into the training data.

In [22]:
X_train_processed = pipeline_X.fit_transform(X_train)
X_test_processed = pipeline_X.transform(X_test)
sv_reg = SVR()
param_grid = [{'kernel':['linear'], 'C': [0.1, 1.0, 10., 100.]},
              {'kernel': ['rbf'], 'C': [0.1, 1.0, 10., 100.], 'gamma': [0.01, 0.1, 1.0, 10]}]
grid_search_leakage = GridSearchCV(sv_reg, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=n_jobs, verbose=2)
grid_search_leakage.fit(X_train_processed, y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=11)]: Using backend LokyBackend with 11 concurrent workers.
[Parallel(n_jobs=11)]: Done  19 tasks      | elapsed:   50.8s
[Parallel(n_jobs=11)]: Done 100 out of 100 | elapsed:  4.3min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                           epsilon=0.1, gamma='auto_deprecated', kernel='rbf',
                           max_iter=-1, shrinking=True, tol=0.001,
                           verbose=False),
             iid='warn', n_jobs=11,
             param_grid=[{'C': [0.1, 1.0, 10.0, 100.0], 'kernel': ['linear']},
                         {'C': [0.1, 1.0, 10.0, 100.0],
                          'gamma': [0.01, 0.1, 1.0, 10], 'kernel': ['rbf']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='neg_mean_squared_error', verbose=2)

In [23]:
negative_mse = grid_search_leakage.best_score_
rmse = np.sqrt(-negative_mse)
rmse

72128.10683248386

In [24]:
grid_search_leakage.best_params_

{'C': 100.0, 'kernel': 'linear'}

### Right approach

To do cross-validation correctly, the entire data processing steps should be re-executed for each fold so that the estimation properly takes into account that the test data is unknown at training.

In [31]:
model_pipe = Pipeline([('feature_pipeline?', pipeline_X),('svr', SVR())])

In [34]:
param_grid = [{'svr__kernel':['linear'], 'svr__C': [0.1, 1.0, 10., 100.]},
              {'svr__kernel': ['rbf'], 'svr__C': [0.1, 1.0, 10., 100.], 'svr__gamma': [0.01, 0.1, 1.0, 10]}]

In [35]:
grid_search = GridSearchCV(model_pipe, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=n_jobs, verbose=2)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=11)]: Using backend LokyBackend with 11 concurrent workers.
[Parallel(n_jobs=11)]: Done  19 tasks      | elapsed:   56.3s
[Parallel(n_jobs=11)]: Done 100 out of 100 | elapsed:  4.5min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('feature_pipeline',
                                        FeatureUnion(n_jobs=None,
                                                     transformer_list=[('numeric',
                                                                        Pipeline(memory=None,
                                                                                 steps=[('selector',
                                                                                         TypeSelector(dtype='float')),
                                                                                        ('imputer',
                                                                                         SimpleImputer(add_indicator=False,
                                                                                                       copy=True,
                                          

In [36]:
negative_mse = grid_search.best_score_
rmse = np.sqrt(-negative_mse)
rmse

72130.35304275915

In [37]:
grid_search.best_params_

{'svr__C': 100.0, 'svr__kernel': 'linear'}

In [44]:
from sklearn.metrics import mean_squared_error

In [45]:
y_test_pred = grid_search.predict(X_test)

In [47]:
np.sqrt(mean_squared_error(y_test_pred, y_test))

69934.21552363789

## Exercise 2

Try replacing `GridSearchCV` with `RandomizedSearchCV`.

In [43]:
param_distributions = {'svr__kernel':['linear', 'rbf'], 'svr__C': reciprocal(1., 10000.), 'svr__gamma': expon(scale=1.)}

In [48]:
random_search = RandomizedSearchCV(model_pipe, param_distributions, cv=5, scoring='neg_mean_squared_error',
                                   n_jobs=n_jobs, verbose=2, n_iter=100, random_state=42)
random_search.fit(X_train, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=11)]: Using backend LokyBackend with 11 concurrent workers.
[Parallel(n_jobs=11)]: Done  19 tasks      | elapsed:   57.4s
[Parallel(n_jobs=11)]: Done 140 tasks      | elapsed:  6.0min
[Parallel(n_jobs=11)]: Done 343 tasks      | elapsed: 14.3min
[Parallel(n_jobs=11)]: Done 500 out of 500 | elapsed: 20.6min finished


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
                   estimator=Pipeline(memory=None,
                                      steps=[('feature_pipeline',
                                              FeatureUnion(n_jobs=None,
                                                           transformer_list=[('numeric',
                                                                              Pipeline(memory=None,
                                                                                       steps=[('selector',
                                                                                               TypeSelector(dtype='float')),
                                                                                              ('imputer',
                                                                                               SimpleImputer(add_indicator=False,
                                                                                                

In [49]:
negative_mse = random_search.best_score_
rmse = np.sqrt(-negative_mse)
rmse

60260.71135638083

In [50]:
random_search.best_params_

{'svr__C': 9502.32435013483,
 'svr__gamma': 0.19349404041660798,
 'svr__kernel': 'rbf'}

In [51]:
y_test_pred = random_search.predict(X_test)

In [52]:
np.sqrt(mean_squared_error(y_test_pred, y_test))

58779.2325235463

# Exercise 3

Try adding a transformer in the preparation pipeline to select only the most important attributes.

### Simple approach

This approach calculates the feature importance based on the best model, as determined before. This ignores that the best model might be different if feature selection was performed during the parameter search.

In [57]:
from sklearn.feature_selection import SelectKBest
from sklearn.inspection import permutation_importance

In [ ]:
model = random_search.best_estimator_

In [ ]:
if model.get_params('kernel') == 'linear':
    feature_importance = model.coef_
else:
    feature_importance = permutation_importance(model, X_train, y_train)

In [ ]:
selection_pipe = Pipeline([('feature_pipeline', pipeline_X),('select', SelectKBest(feature_importance), ('svr', SVR())])

In [ ]:
selection_pipe.fit(X_train, y_train)

### Comprehensive approach

The SVR with a kernel other then linear does not have any method to determine the importance of features. A general method to guess this would be to use the permutation importance. As this calculation requires cross-validation, the procedure is computationally quite demanding.

In [29]:
class selectSVR(SVR):
    def fit(self, X, y, sample_weight=None):
        super().fit(X, y, sample_weight)
        if clf.get_params()['kernel'] != 'linear':
            perm_imp = permutation_importance(self, X.todense(), y, n_jobs=1)
            self.permutation_importances_ = perm_imp['mean_permutation_importance']
        return self
    @property
    def feature_importances_(self):
        if clf.get_params()['kernel'] != 'linear':
            return self.permutation_importances_
        else:
            return self.coef_

In [49]:
class fitSVR(SVR):
    def __init__(self):
        super().__init__(self, self.named_steps['select'].get_params()['estimator'].get_params())
    def get_params(self, deep=True):
        super().get_params(self, deep)
    def set_params(self, **params):
        super().set_params(self, **params)

In [ ]:
from sklearn.feature_selection import SelectFromModel

In [33]:
selection_pipe = Pipeline([('feature_pipeline', pipeline_X),('select', SelectFromModel(selectSVR())),('svr', fitSVR())])

In [ ]:
param_distributions = {'select__kernel':['linear', 'rbf'], 'select__C': reciprocal(1., 10000.), 'select__gamma': expon(scale=1.)}

# Exercise 4

Try creating a single pipeline that does the full data preparation plus the final prediction.

Already done above

# Exercise 5

Automatically explore some preparation options using `GridSearchCV`.

In [58]:
pipeline_num = Pipeline([
    ('selector', TypeSelector('float')),
    ('imputer', SimpleImputer(strategy='median')),
    ('attr_adder', CombinedAttributesAdder()),
    ('scaler', RobustScaler())
])
pipeline_cat = Pipeline([
    ('selector', TypeSelector('object')),
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder())
])
pipeline_X = FeatureUnion([('numeric', pipeline_num), ('categorical', pipeline_cat)])

In [59]:
selection_pipe = Pipeline([('feature_pipeline', pipeline_X),('select', SelectKBest(feature_importance)), ('svr', SVR())])

NameError: name 'feature_importance' is not defined

In [63]:
from sklearn.preprocessing import OrdinalEncoder

In [64]:
prep_grid = {'feature_pipeline__numeric__imputer__strategy': ['median', 'mean', 'most_frequent'],
             'feature_pipeline__numeric__attr_adder__add_rooms_per_household': [True, False],
              'feature_pipeline__numeric__attr_adder__add_population_per_household': [True, False],
              'feature_pipeline__numeric__attr_adder__add_bedrooms_per_room': [True, False],
              'feature_pipeline__categorical__encoder': [OneHotEncoder(), OrdinalEncoder()],
             }

In [66]:
param_grid = [{'svr__kernel':['linear'], 'svr__C': [100., 1000., 10000, 100000.]}.update(prep_grid),
              {'svr__kernel': ['rbf'], 'svr__C': [100., 1000., 10000, 100000.], 'svr__gamma': [0.01, 0.1, 1.0]}.update(prep_grid)]

In [ ]:
grid_search = GridSearchCV(model_pipe, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=n_jobs, verbose=2)
grid_search.fit(X_train, y_train)